# 08. Convolutional neural network

In [1]:
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv3D,MaxPool3D,Flatten,Dense,Dropout,Conv2D,MaxPool2D

/Users/jasonola/opt/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jasonola/opt/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jasonola/opt/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jasonola/opt/an

Let's take the raw data this time

In [2]:
with np.load("raw_train.npz",allow_pickle=False) as train_npz:
    train_data = dict(train_npz.items())

with np.load("raw_valid.npz",allow_pickle=False) as valid_npz:
    valid_data = dict(valid_npz.items())

with np.load("raw_test.npz",allow_pickle=False) as test_npz:
    test_data = dict(test_npz.items())

In [3]:
X_train,y_train = train_data.values()
X_valid,y_valid = valid_data.values()
X_test,y_test = test_data.values()

In [4]:
X_train = np.concatenate((X_train,X_valid))
y_train = np.concatenate((y_train,y_valid))

We can now create a convolutional model, data is already scaled between 0-1.  
I'll make a similar model as in the course but implement it with keras.  
I make 2 convolutional layers with maxpooling and flatten in between as well as a drop out layer to prevent overfitting

In [5]:
y_train.max()+1

6.0

In [9]:
X_train.shape

(420, 299, 299, 3)

In [6]:
input_shape = X_train.shape[-3:]
no_logits = y_train.max()+1

In [7]:
cnn = keras.Sequential()
cnn.add(Conv2D(filters=16,kernel_size=(5,5),strides=(2,2),padding="same",activation="relu",input_shape=input_shape))
cnn.add(MaxPool2D(strides=(2,2),padding="same"))
cnn.add(Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
cnn.add(MaxPool2D(strides=(2,2),padding="same"))
cnn.add(Flatten())
cnn.add(Dropout(0.5))
cnn.add(Dense(units=256,activation="relu",kernel_regularizer=keras.regularizers.l2(0.01)))
cnn.add(Dense(units=no_logits,activation="softmax"))
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 23104)             0         
_________________________________________________________________
dropout (Dropout)            (None, 23104)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               5914880   
__________

Now that we have a model let's fit it.

In [8]:
cnn.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["acc"])
early_stopping = keras.callbacks.EarlyStopping(patience=12)

In [9]:
history = cnn.fit(X_train,y_train,batch_size=32,epochs=100,validation_split=0.2,callbacks=[early_stopping])

Train on 336 samples, validate on 84 samples
Epoch 1/100
336/336 [==============================] - 9s 26ms/step - loss: 5.9152 - acc: 0.1756 - val_loss: 4.1371 - val_acc: 0.2262
Epoch 2/100
336/336 [==============================] - 4s 12ms/step - loss: 3.6345 - acc: 0.2738 - val_loss: 3.0786 - val_acc: 0.2857
Epoch 3/100
336/336 [==============================] - 3s 10ms/step - loss: 2.8912 - acc: 0.3006 - val_loss: 2.5382 - val_acc: 0.4524
Epoch 4/100
336/336 [==============================] - 3s 10ms/step - loss: 2.3962 - acc: 0.3125 - val_loss: 2.1920 - val_acc: 0.3452
Epoch 5/100
336/336 [==============================] - 3s 10ms/step - loss: 2.1046 - acc: 0.3482 - val_loss: 2.3859 - val_acc: 0.1786
Epoch 6/100
336/336 [==============================] - 3s 10ms/step - loss: 1.9233 - acc: 0.3661 - val_loss: 1.9535 - val_acc: 0.3810
Epoch 7/100
336/336 [==============================] - 3s 10ms/step - loss: 1.7611 - acc: 0.4732 - val_loss: 1.8498 - val_acc: 0.3929
Epoch 8/100
336/3

In [10]:
mean_val_acc = np.mean(history.history["val_acc"][-5:])
print("Mean validation accuracy (last 5) : {:.1f}%".format(100*mean_val_acc))

Mean validation accuracy (last 5) : 39.5%


We see that we get an accuracy below 50% which is a little dissapointing but knowing that we worked with high-level features until now, maybe it's not that bad. Also, I think that if we had more data than just a few hundreds, we could have better results too.

In [11]:
(cnn_test_loss,cnn_test_acc) = cnn.evaluate(X_test,y_test)

50/50 [==============================] - 0s 7ms/step


In [12]:
cnn_test_acc = round(cnn_test_acc*100,1)
print("Test accuracy cnn : {:.1f}%".format(cnn_test_acc))

Test accuracy cnn : 40.0%


In [13]:
%store cnn_test_acc

Stored 'cnn_test_acc' (float64)
